In [ ]:
 # Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount= True)


Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

from PIL import Image
import os

custom_image_path = '/content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/Input'

# Image transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the custom dataset
try:
    dataset = ImageFolder(root=custom_image_path, transform=transform)
    print("Dataset loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please check if the specified directory exists and has the correct structure.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(self.act1(self.conv1(x)))
        x = self.pool(self.act1(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except the batch dimension
        x = self.act1(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, criterion, and optimizer
model = SimpleCNN(num_classes=len(dataset.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
train_losses = []
val_losses = []
l=[]

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

    model.eval()
    val_loss = 0.0
    val_corrects = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)

    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_corrects.double() / len(val_loader.dataset)

    num=val_accuracy.tolist() #'0.29'--> tensor to string(float)
    l.append(float("{:.4f}".format(num)))  #string-->float
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {num:.4f}")

    train_losses.append(train_loss)
    val_losses.append(val_loss)





Dataset loaded successfully!
Epoch 1/10, Train Loss: 3.7432, Val Loss: 0.4940, Val Accuracy: 0.8525
Epoch 2/10, Train Loss: 0.2738, Val Loss: 0.1986, Val Accuracy: 0.9016
Epoch 3/10, Train Loss: 0.0915, Val Loss: 0.2741, Val Accuracy: 0.9016
Epoch 4/10, Train Loss: 0.0681, Val Loss: 0.1888, Val Accuracy: 0.8852
Epoch 5/10, Train Loss: 0.0231, Val Loss: 0.3025, Val Accuracy: 0.9262
Epoch 6/10, Train Loss: 0.0248, Val Loss: 0.2864, Val Accuracy: 0.8934
Epoch 7/10, Train Loss: 0.0057, Val Loss: 0.2225, Val Accuracy: 0.9016
Epoch 8/10, Train Loss: 0.0024, Val Loss: 0.2228, Val Accuracy: 0.9098
Epoch 9/10, Train Loss: 0.0012, Val Loss: 0.2269, Val Accuracy: 0.9098
Epoch 10/10, Train Loss: 0.0007, Val Loss: 0.2413, Val Accuracy: 0.9098


In [ ]:
# Define the path where you want to save your model
model_save_path = '/content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/models'
os.makedirs(model_save_path, exist_ok=True)  # Create the directory if it doesn't exist

# Save the model
torch.save(model.state_dict(), os.path.join(model_save_path, 'suman sir_cnn_model.pth'))

# Optionally, you can save the entire model including architecture and parameters
# torch.save(model, os.path.join(model_save_path, 'entire_model.pth'))

# Now you have saved your model in the specified folder in Google Drive


In [ ]:
# Define the path to the saved model
saved_model_path = '/content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/models/suman sir_cnn_model.pth'

# Initialize a new instance of your model
loaded_model = SimpleCNN(num_classes=len(dataset.classes)).to(device)

# Load the saved model state dictionary
loaded_model.load_state_dict(torch.load(saved_model_path))
print("Model loaded successfully from:", saved_model_path)

# Make sure to set the model to evaluation mode if you're not planning to further train it
loaded_model.eval()


Model loaded successfully from: /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/models/suman sir_cnn_model.pth


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [ ]:
import cv2

def extract_frames(video_path, output_folder, interval_seconds):
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    print(frame_rate)
    interval_frames = int(frame_rate * interval_seconds)
    frame_count = 0
    count=0


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % interval_frames == 0:
            output_file = f"{output_folder}/frame_{count}.jpg"
            cv2.imwrite(output_file, frame)
            print(f"Frame {frame_count} saved as {output_file}")
            count=count+1

        frame_count += 1
    cap.release()
    return count
if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/videos/suman sir.mp4"  # Specify the path to your video
    output_folder = "/content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir"  # Specify the folder where frames will be saved
    interval_seconds = 60  # Interval in seconds

    count=extract_frames(video_path, output_folder, interval_seconds)


50.0
Frame 0 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_0.jpg
Frame 3000 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_1.jpg
Frame 6000 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_2.jpg
Frame 9000 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_3.jpg
Frame 12000 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_4.jpg
Frame 15000 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_5.jpg
Frame 18000 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_6.jpg
Frame 21000 saved as /content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir/frame_7.jpg
Frame 2400

In [ ]:
from PIL import Image
import os
import math
# Directory containing the images
directory_path = r"/content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/video_frames/suman_sir"
confident=0
confused=0
# Get all files in the directory
image_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Loop through each image
for image_path in image_paths:
    # Load the image and preprocess it
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    loaded_model.eval()

    # Make the prediction
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)

    # Map the predicted index to the class label
    class_labels = ['confused', 'confident']
    prediction = class_labels[predicted.item()]
    if prediction=="confident":
      confident+=1
    else:
      confused+=1
    print(f"{os.path.basename(image_path)} is classified as: {prediction}")
print("Confident",confident)
print("COnfused",confused)

frame_0.jpg is classified as: confused
frame_1.jpg is classified as: confident
frame_2.jpg is classified as: confident
frame_3.jpg is classified as: confident
frame_4.jpg is classified as: confident
frame_5.jpg is classified as: confident
frame_6.jpg is classified as: confident
frame_7.jpg is classified as: confused
frame_8.jpg is classified as: confident
frame_9.jpg is classified as: confident
frame_10.jpg is classified as: confident
frame_11.jpg is classified as: confident
frame_12.jpg is classified as: confident
frame_13.jpg is classified as: confident
frame_14.jpg is classified as: confident
frame_15.jpg is classified as: confused
frame_16.jpg is classified as: confident
frame_17.jpg is classified as: confident
frame_18.jpg is classified as: confident
frame_19.jpg is classified as: confident
frame_20.jpg is classified as: confident
frame_21.jpg is classified as: confident
frame_22.jpg is classified as: confident
frame_23.jpg is classified as: confident
frame_24.jpg is classified as

In [ ]:
total_frames=confident+confused
print(total_frames)
per_conf=confident/total_frames
print("Your score is:",per_conf*100,"%")
threshold_value=float(input("Enter the threshold between 0.5 to 1:->"))
if per_conf>=threshold_value:
  print("Yes the teacher is confident!")
else:
  print("The teacher is less confident!")

46
Enter the threshold between 0.5 to 1:->0.869
Yes the teacher is confident!


In [ ]:
!pip install tensorflow lite

In [ ]:
!pip install onnx
!pip install onnx-tf
!pip install onnx onnx-tf
!pip install onnx onnx-tf tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 22.5 MB/s eta 0:00:00


In [ ]:
!pip install onnx onnxruntime tensorflow-addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
from torch.autograd import Variable
import torch.onnx
import torchvision
import torch
import torchvision.models as models

In [ ]:
# Load the pretrained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)

# Assuming the model is already trained and saved in the provided path
state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/Final Year Project/Data Sets/models/suman sir_cnn_model.pth')
model.load_state_dict(state_dict, strict=False)

# Set the model to evaluation mode
model.eval()

# Define dummy input with the same shape as the actual input
dummy_input = Variable(torch.randn(1, 3, 224, 224))

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "suman_sir_cnn_model.onnx", verbose=True, input_names=["input"], output_names=["output"])

# Convert the ONNX model to TFLite format using the onnx-tf package
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("suman_sir_cnn_model.onnx")
tf_rep = prepare(onnx_model)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import onnx
import tensorflow as tf
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load("suman_sir_cnn_model.onnx")

# Convert ONNX model to TensorFlow format
tf_rep = prepare(onnx_model)

# Save the TensorFlow model to a directory
tf_saved_model_dir = "tf_saved_model"
tf_rep.export_graph(tf_saved_model_dir)

# Convert the TensorFlow SavedModel to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model(tf_saved_model_dir)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open("suman_sir_cnn_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
!pip install --upgrade google-api-python-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
